In [1]:
#|default_exp examples

In [2]:
#|export
from titrations.basics import *
from titrations.titrations2 import *

# Beta Blockers

## Dosing Ladder

In [3]:
#|export

metoprolol_succinate = Ingredient("metoprolol succinate")
carvedilol = Ingredient("carvedilol")
bisoprolol = Ingredient("bisoprolol")

beta_blocker_ladder = DosingLadder({
    metoprolol_succinate.name: [
        Medication(metoprolol_succinate, "12.5 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "25 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "50 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "100 mg", "PO", "daily"),
    ],
    carvedilol.name: [
        Medication(carvedilol, "3.125 mg", "PO", "BID"),
        Medication(carvedilol, "6.25 mg", "PO", "BID"),
        Medication(carvedilol, "12.5 mg", "PO", "BID"),
        Medication(carvedilol, "25 mg", "PO", "BID"),
    ],
    bisoprolol.name: [
        Medication(bisoprolol, "1.25 mg", "PO", "BID"),
        Medication(bisoprolol, "2.5 mg", "PO", "BID"),
        Medication(bisoprolol, "5 mg", "PO", "BID"),
        Medication(bisoprolol, "10 mg", "PO", "BID"),
    ]
})

## Rules

In [4]:
#|export
no_pacemaker = Rule('has_pacemaker', 'eq', False)

hypotension = TitrationLimitingRule('SBP', 'lt', 100)
bradycardia = ConditionTitrationLimitingRule('HR' , 'lt', 60, condition=no_pacemaker)
decompensation = TitrationLimitingRule('decompensated', 'eq', True)
symptoms = TitrationLimitingRule('symptomatic', 'eq', True)
av_block = TitrationLimitingRule('av_block', 'eq', True)

## Titrator

In [5]:
#|export
class BetaBlockerTitrator(Titrator):
    dosing_ladder = beta_blocker_ladder
    rules = [
        hypotension,
        bradycardia,
        decompensation,
        symptoms,
        av_block,
    ]

# RAASi

## Dosing Ladder

In [6]:
#|export

sacubitril_valsartan = Ingredient("sacubitril/valsartan")
lisinopril = Ingredient("lisinopril")
losartan = Ingredient("losartan")

raasi_ladder = DosingLadder({
    sacubitril_valsartan.name: [
        # NOTE: the lengths of these subladders are unequal
        Medication(sacubitril_valsartan, "24-26 mg", "PO", "BID"),
        Medication(sacubitril_valsartan, "49-51 mg", "PO", "BID"),
        Medication(sacubitril_valsartan, "97-103 mg", "PO", "BID"),
    ],
    lisinopril.name: [
        Medication(lisinopril, "5 mg", "PO", "daily"),
        Medication(lisinopril, "10 mg", "PO", "daily"),
        Medication(lisinopril, "20 mg", "PO", "daily"),
        Medication(lisinopril, "40 mg", "PO", "daily"),
    ],
    losartan.name: [
        Medication(losartan, "12.5 mg", "PO", "daily"),
        Medication(losartan, "25 mg", "PO", "daily"),
        Medication(losartan, "50 mg", "PO", "daily"),
        Medication(losartan, "100 mg", "PO", "daily"),
    ]
})

## Rules

In [7]:
#|export
low_egfr = TitrationLimitingRule('eGFR', 'lt', 30)
hyperkalemia = TitrationLimitingRule('K', 'gt', 5)
hypotension
symptoms
# angioedema = ClassLimitingRule()  # TODO

symptomatic eq True

## Titrator

In [8]:
#|export
class RAASiTitrator(Titrator):
    dosing_ladder = raasi_ladder
    rules = [
        low_egfr,
        hyperkalemia,
        hypotension,
        symptoms,
        # angioedema,
    ]

# SGLT2i's

## Dosing Ladder

In [9]:
#|export
dapagliflozin = Ingredient("dapagliflozin")
empagliflozin = Ingredient("empagliflozin")

sglt2i_ladder = DosingLadder({
    dapagliflozin.name: [
        Medication(dapagliflozin, "10 mg", "PO", "daily"),
    ],
    empagliflozin.name: [
        Medication(empagliflozin, "12.5 mg", "PO", "daily"),
    ],
})

## Rules

In [10]:
#|export
low_egfr # this is the same rule as above, this is an example of reusing a rule
severe_gu_infxns = ClassLimitingRule('severe_gu_infxns', 'eq', True)
has_type_1_diabetes = ClassLimitingRule('has_type_1_diabetes', 'eq', True)
has_type_2_diabetes_on_insulin = NonLimitingRule('has_type_2_diabetes_on_insulin', 'eq', True)

## Titrator

In [11]:
#|export
class SGLT2iTitrator(Titrator):
    dosing_ladder = sglt2i_ladder
    rules = [
        low_egfr,
        severe_gu_infxns,
        has_type_1_diabetes,
        has_type_2_diabetes_on_insulin,
    ]

## Tests

In [12]:
p = Patient(eGFR=20, severe_gu_infxns=False,
            has_type_1_diabetes=False, has_type_2_diabetes_on_insulin=False,
            medications=[Medication(empagliflozin, "12.5 mg", "PO", "daily")])
t = SGLT2iTitrator(p)
t.evaluate()

In [13]:
t.can_advance

False

In [14]:
t.satisfied_rules

[eGFR lt 30]

In [15]:
t.recommended_actions

# MRA's

## Dosing Ladder

In [16]:
#|export

spironolactone = Ingredient("spironolactone")
eplerenone = Ingredient("eplerenone")

mra_ladder = DosingLadder({
    spironolactone.name: [
        Medication(spironolactone, "12.5 mg", "PO", "daily"),
        Medication(spironolactone, "25 mg", "PO", "daily"),
    ],
    eplerenone.name: [
        Medication(eplerenone, "12.5 mg", "PO", "daily"),
        Medication(eplerenone, "25 mg", "PO", "daily"),
    ],
})

## Rules

In [17]:
low_egfr # this is the same rule as above, this is an example of reusing a rule
hyperkalemia # this is the same rule as above, this is an example of reusing a rule
# on_potassium_supp = NonLimitingRule('on_potassium_supp', 'eq', True)
# spiro_side_effects = NonLimitingRule('spiro_side_effects', 'eq', True)

K gt 5

## Titrator

In [18]:
#|export
class MRATitrator(Titrator):
    dosing_ladder = mra_ladder
    rules = [
        low_egfr,
        hyperkalemia,
        # TODO
    ]

# Export

In [20]:
# from nbdev.export import nb_export

# nb_export('examples.ipynb')